need a spline library with the right boundary conditions.


In [ ]:
using Interpolations, Dierckx, PyPlot

In [ ]:
x = linspace(0, 1, 10) |> collect 
y = rand(10)
y[end] = 0.0

In [ ]:
xx = linspace(0, 2, 100)
yy = sin(xx * pi).^3 .* (xx .<= 1)
xknots = [linspace(0.1, 1, 10); 1.01; 1.02]
spl = Spline1D(xx, yy, xknots, bc = "zero")
t = linspace(0.2, 1.2, 200) |> collect
plot(t, spl(t), "b-", t, derivative(spl, t), "r-")
axis([0.0, 1.2, -0.5, 1.1])

In [ ]:
?Dierckx

In [ ]:
spli = interpolate(y, BSpline(Linear), OnGrid)

In [ ]:
?interpolate

In [ ]:
nx = 10
f(x) = sin(2pi/(nx-1) * (x-1))

xcoarse = 1:nx
ycoarse = f(xcoarse);

xfine = 1:.1:xcoarse[end]
yfine = f(xfine);


In [ ]:
filename = "/Users/ortner/Downloads/ffe.plt.txt"

In [ ]:
file = open(filename, "r")

In [ ]:
data = readdlm(filename, Float64)
x, y = data[:,1], data[:,2];

In [ ]:
plot(x, y)
axis([0.0, 6.0, -1.0, 2.0])

In [ ]:
spl = Spline1D(x, y; bc = "zero", s = 1e-3)
spl.c[end-3:end] = 0.0
t = linspace(1.0, 6.0, 200) |> collect
plot(t, spl(t), "b-", t, derivative(spl, t), "r-")
axis([3, 6, -1e-2, 1e-2])


In [ ]:
spl.c